# PROV Templates in Python

     Author: Stephan Kindermann
     Affiliation: DKRZ
     Community: ENES (Earth System Sciences)
     Version: 0.2 (July 2018)
    
**Motivation:**
* The adoption of PROV templates in ENES community workflows is hindered by the following aspects:

    * template generation in ENES oftenly needs to be tool based (template structure depends on project/experiment configuration) 
    * template instanciation in ENES oftenly is done in a scripting environment (most oftenly using python)
    * template expansion currently is only supported by the **provconvert** java tool
    * the transition from Java/provconvert to Python/prov is not fully supported (no import of provconvert output in prov supported) 
   
* The sharing PROV adoption narratives is well supported by using jupyter notebooks and python.
* Core infrastructure services in ENES are implemented using python. 

Therfore: 
* The java based **provconvert** tool thus is difficult to exploit in ENES use cases
* Using prov templates is by now only usefull for "documentation" purposes - no impact on community approaches. 
* To have an impact on community approaches in the short term simple python based wrappers are needed which can be integrated in our community workflows.  

**Approach taken**
A simple template instantiation package (called provtemplates) was implemented supporting the instantiation of prov templates in python.  
This approach on the one hand side allows to generate and use PROV templates in python and which on the other hand side allows for pure python based template instantiations.

Drawback (by now) is that the elaborated prov expansion algorithm is not yet implemented fully. But expansion rules can implemented easyly explicitely in python when needed (and later integrated into the python package). To make the expansion explicit on the basis of python can also seen as an advantage as the community PROV adopters don't need to dig into the expansion algorithm implemented as part of provconvert (and eventual errors therein).

The approach taken is illustrated in the following:   
* PROV templates (being standard PROV documents) are generated in Python based on the prov library alongside PROV template instances.
* A very simple instantiation algorithm is used to instantiate templates based on dictionaries containing the variable settings .. this instantiation algorithm can be stepwise expanded in the future


## Generate a PROV template
PROV templates are generated in functions with all RROV variables as parameters.
This function is called with prov template variable names to generate prov templates. When called with instances the result is an prov document corresponding to the instantiated prov template.

In the following the approach is illustrated based on a concrete examplec (corresponding to the first example in the provconvert tutorial). 

### Initialize empty PROV template with namespaces needed

In [ ]:
# Define namespaces used and generate a new empty 
# python prov template instance
# Define the variable settings in the template as a dictionary

from provtemplates import provconv
import prov.model as prov
import six
import itertools


ns_dict = {
    'prov':'http://www.w3.org/ns/prov#',
    'var':'http://openprovenance.org/var#>',
    'vargen':'http://openprovenance.org/vargen#',
    'tmpl':'http://openprovenance.org/tmpl#',
    'foaf':'http://xmlns.com/foaf/0.1/',
    'ex': 'http://example.org/',
    'orcid':'http://orcid.org/',

    #document.set_default_namespace('http://example.org/0/')
    'rdf':'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'rdfs':'http://www.w3.org/2000/01/rdf-schema#',
    'xsd':'http://www.w3.org/2001/XMLSchema#',
    'ex1': 'http://example.org/1/',
    'ex2': 'http://example.org/2/'
}


instance_dict = {
    'var:author':'orcid:0000-0002-3494-120X',
    'var:value':'A Little Provenance Goes a Long Way',
    'var:name':'Luc Moreau',
    'var:quote':'ex:quote1'
}

doc0 = provconv.set_namespaces(ns_dict,prov.ProvDocument())


### Generate an example prov template and print (and store) it's provn representation

In [ ]:
doc1 = provconv.save_and_show(doc0,'/home/stephan/test/xxxx.provn')

## Instantiate prov template

the instantiation is done via the provconf.instantate_template function

for comparison the template file as well as the instance are printed

In [ ]:
new = provconv.instantiate_template(doc1,instance_dict)
print(doc1.get_provn())
print(new.get_provn())

In [ ]:
%matplotlib  inline
doc1.plot()
new.plot()

## Compare with original ProvToolbox example: 

In [ ]:
# %load Downloads/ProvToolbox-Tutorial4-0.7.0/src/main/resources/template1.provn

document

  prefix var <http://openprovenance.org/var#>
  prefix vargen <http://openprovenance.org/vargen#>
  prefix tmpl <http://openprovenance.org/tmpl#>

  prefix foaf <http://xmlns.com/foaf/0.1/>
  
  bundle vargen:bundleId
    entity(var:quote, [prov:value='var:value'])
    entity(var:author, [prov:type='prov:Person', foaf:name='var:name'])
    wasAttributedTo(var:quote,var:author)
  endBundle

endDocument


In [ ]:
# take same instantiation as in the tutorial:
# %load Downloads/ProvToolbox-Tutorial4-0.7.0/src/main/resources/binding1.ttl
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix tmpl: <http://openprovenance.org/tmpl#> .
@prefix var: <http://openprovenance.org/var#> .
@prefix ex: <http://example.com/#> .

var:author a prov:Entity;
           tmpl:value_0 <http://orcid.org/0000-0002-3494-120X>.
var:name   a prov:Entity;
           tmpl:2dvalue_0_0 "Luc Moreau".
var:quote  a prov:Entity;
           tmpl:value_0 ex:quote1.
var:value  a prov:Entity;
           tmpl:2dvalue_0_0 "A Little Provenance Goes a Long Way".



## Instantiate PROV template using provconvert and compare results

In [ ]:
!provconvert -infile test/template1.provn -bindings test/binding1.ttl -outfile test/doc1.provn

In [ ]:
# %load test/doc1.provn
document
bundle uuid:672b425e-a7db-470f-8653-7318d9ae8ec1
prefix foaf <http://xmlns.com/foaf/0.1/>
prefix pre_0 <http://orcid.org/>
prefix ex <http://example.com/#>
prefix uuid <urn:uuid:>

entity(ex:quote1,[prov:value = "A Little Provenance Goes a Long Way" %% xsd:string])
entity(pre_0:0000-0002-3494-120X,[prov:type = 'prov:Person', foaf:name = "Luc Moreau" %% xsd:string])
wasAttributedTo(ex:quote1, pre_0:0000-0002-3494-120X)
endBundle
endDocument


In [ ]:
!provconvert -infile test/doc1.provn -outfile test/doc1.png


In [ ]:
!provconvert -infile test/template1.provn -outfile test/template1.png

In [ ]:
# %load Downloads/ProvToolbox-Tutorial4-0.7.0/target/doc1.provn
document
bundle uuid:4c7236d5-6420-4a88-b192-6089e27aa88e
prefix foaf <http://xmlns.com/foaf/0.1/>
prefix pre_0 <http://orcid.org/>
prefix ex <http://example.com/#>
prefix uuid <urn:uuid:>

entity(ex:quote1,[prov:value = "A Little Provenance Goes a Long Way" %% xsd:string])
entity(pre_0:0000-0002-3494-120X,[prov:type = 'prov:Person', foaf:name = "Luc Moreau" %% xsd:string])
wasAttributedTo(ex:quote1, pre_0:0000-0002-3494-120X)
endBundle
endDocument


# Show provconvert result to compare

![in](./test/doc1.png)
![out](./test/doc2.png)